<a href="https://colab.research.google.com/github/sophie-w/COVID-MDS/blob/master/MedCAT_Tutorial_%7C_Part_4_1_ByteLevelBPETokenizer_and_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --upgrade medcat

     |████████████████████████████████| 133kB 9.8MB/s 
     |████████████████████████████████| 358kB 16.7MB/s 
     |████████████████████████████████| 15.3MB 268kB/s 
     |████████████████████████████████| 225kB 43.6MB/s 
     |████████████████████████████████| 10.4MB 23.3MB/s 
     |████████████████████████████████| 2.1MB 29.9MB/s 
     |████████████████████████████████| 24.2MB 1.7MB/s 
     |████████████████████████████████| 27.4MB 159kB/s 
     |████████████████████████████████| 245kB 50.9MB/s 
     |████████████████████████████████| 122kB 51.4MB/s 
     |████████████████████████████████| 1.1MB 35.2MB/s 
     |████████████████████████████████| 901kB 15.4MB/s 
     |████████████████████████████████| 3.3MB 21.2MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.

In [ ]:
from tokenizers import ByteLevelBPETokenizer
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

In [ ]:
DATA_DIR = "./data/"

In [ ]:
!mkdir ./data
!mkdir ./models
!wget https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/noteevents.csv -P ./data/

--2021-06-15 00:02:04--  https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/noteevents.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171226 (6.8M) [text/plain]
Saving to: ‘./data/noteevents.csv’

noteevents.csv      100%[===================>]   6.84M  44.3MB/s    in 0.2s    

2021-06-15 00:02:04 (44.3 MB/s) - ‘./data/noteevents.csv’ saved [7171226/7171226]



### Meta Annotations with MedCAT

To train meta-annotations (e.g. Experiencer, Negation...) we need two additional models:
- Tokenizer: to tokenize the text
- Embeddings: Word2Vec or any other type of embeddings that will be used for meta annotations. 

For meta-annotations we will use a custom BiLSTM model with simulated attention that works very well with sub-word tokenizers and embeddings creating using Word2Vec or BERT (for simplicity we will use w2v here). All of this is also available for download (check next tutorial) and we only need to rebuild the tokenizer/embeddings if our use-case is from a very specific domain. 

In [ ]:
# To train the tokenizer we will use all the data we have from our dummy dataset.
df = pd.read_csv(DATA_DIR + "noteevents.csv")
df.head()

,Unnamed: 0,subject_id,chartdate,category,text
0,0,0,01/01/2086,Urology,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
1,1,0,01/01/2086,Emergency Room Reports,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
2,2,0,01/01/2086,General Medicine,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."
3,3,0,01/01/2086,General Medicine,"CHIEF COMPLAINT:, Followup on hypertension an..."
4,4,0,01/01/2086,Consult - History and Phy.,"CHIEF COMPLAINT: , Blood in urine.,HISTORY OF ..."


In [ ]:
# The tokenizers from huggingface require us to save all the text used for 
#training into one/multiple text files.
f = open(DATA_DIR + "tok_data.txt", 'w')
for text in df['text'].values:
  #We'll remove new lines, so that we have one document in one line
  text = text.strip().replace("\n", ' ')
  f.write(text.lower()) # Lowercase text to remove noise
  f.write("\n")
f.close()

In [ ]:
# Create, train and save the tokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(DATA_DIR + "tok_data.txt")
tokenizer.save("./models/bbpe")

In [ ]:
# Now we tokenize all the text we have and train word2vec
f = open(DATA_DIR + "tok_data.txt", 'r')
# Note that if you have a very large dataset, use iterators that
#read the text line by line from the file, do not load the whole file
#into memory.
data = []
for line in f:
  data.append(tokenizer.encode(line).tokens)
w2v = Word2Vec(data, size=300, min_count=1)

In [ ]:
# Check is word2vec trained, Ġ - for this tokenizer denotes start of word (a space)
w2v.most_similar('Ġcancer')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('Ġcolon', 0.7464339733123779),
 ('Ġmetastatic', 0.7170583605766296),
 ('Ġbreast', 0.681978702545166),
 ('Ġmesothelioma', 0.663028359413147),
 ('Ġcarcinoma', 0.6485821008682251),
 ('Ġfather', 0.6427798867225647),
 ('Ġaugmentation', 0.6409773826599121),
 ('Ġca', 0.6338627338409424),
 ('Ġfamily', 0.619067907333374),
 ('Ġmother', 0.6181211471557617)]

In [ ]:
# Now we just have to create the embeddings matrix
embeddings = []
for i in range(tokenizer.get_vocab_size()):
  word = tokenizer.id_to_token(i)
  if word in w2v.wv:
    embeddings.append(w2v.wv[word])
  else:
    # Assign a random vector if the word was not frequent enough to receive
    #an embedding
    embeddings.append(np.random.rand(300))

In [ ]:
# Save the embeddings
np.save(open("./models/embeddings.npy", 'wb'), np.array(embeddings))